In [ ]:
"""
Reference:
https://sentinelhub-py.readthedocs.io/en/latest/examples/process_request.html
https://sentinelhub-py.readthedocs.io/en/latest/examples/data_search.html#Combine-Catalog-API-with-Process-API
"""

In [ ]:
import datetime as dt

from sentinelhub import (
    BBox,
    bbox_to_dimensions,
    CRS,
    DataCollection,
    filter_times,  
    MimeType,
    MosaickingOrder,
    SentinelHubCatalog,
    SentinelHubRequest,
    SHConfig
    )

In [ ]:
# Load the credentials

from sentinelhub import SHConfig

config = SHConfig()

if not config.sh_client_id or not config.sh_client_secret:
    print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")

In [ ]:
data_collection = DataCollection.SENTINEL2_L1C

In [ ]:
bounding_box_wgs84 = (12.993805, 42.004331, 13.127701, 42.120625)

In [ ]:
resolution=20
search_bbox = BBox(bounding_box_wgs84, crs=CRS.WGS84).buffer(0.2)
bbox_size = bbox_to_dimensions(search_bbox, resolution)
search_time_interval = (dt.datetime(2022, 1, 1), dt.datetime(2022, 12, 31))

In [ ]:
# Get unique acquisitions dates
catalog = SentinelHubCatalog(config=config)
search_iterator = catalog.search(
    DataCollection.SENTINEL2_L1C,
    bbox=search_bbox,
    time=search_time_interval,
    filter="eo:cloud_cover < 5",
    fields={"include": ["id", "properties.datetime", "properties.eo:cloud_cover"], "exclude": []},
)
time_difference = dt.timedelta(hours=1)
all_timestamps = search_iterator.get_timestamps()
unique_acquisitions = filter_times(all_timestamps, time_difference)

In [ ]:
evalscript = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12", "CLM"],
                units: "DN"
            }],
            output: {
                bands: 14,
                sampleType: "INT16"
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B01,
                sample.B02,
                sample.B03,
                sample.B04,
                sample.B05,
                sample.B06,
                sample.B07,
                sample.B08,
                sample.B8A,
                sample.B09,
                sample.B10,
                sample.B11,
                sample.B12,
                sample.CLM];
    }
"""

In [ ]:
# Download the data

for timestamp in unique_acquisitions:
    request = SentinelHubRequest(
        data_folder="output/s2_data",
        evalscript=evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                time_interval=(timestamp - time_difference, timestamp + time_difference),
                maxcc=0.05
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=search_bbox,
        size=bbox_size,
        config=config,
    )
    request.save_data()